<br>

<p style='text-align: right;'> Birkan Emrem </p>
<p style='text-align: right;'> 19.11.2025 </p>
<p style='text-align: right;'> Density Functional Tight Binding (DFTB) and Applications with DFTB+  </p>

# Density Functional Tight Binding (DFTB) and Applications with DFTB+

<hr style="border: 1.5px solid gray;">

## Introduction & Theory

- Why do we need methods faster than DFT?
- The trade-off between accuracy vs computational cost
- Where DFTB fits: „between classical force fields and full DFT“

<img src="Figures/Picture1.png" style="height:300px">

#### Ideal for:
- Large-scale molecular dynamics
- Materials discovery and screening
- Studying reactivity in complex systems

<hr style="border: 1.5px solid gray;">

## Theoretical Background — DFTB

Density Functional Tight-Binding (DFTB):
- A computationally efficient approximation to DFT
- Ideal for large systems (biomolecules, MD simulations)

<img src="Figures/Picture2.png" style="height:200px">

$$
{\huge  E_{DFTB} = E_{BS} + E_{coul} + E_{rep}}
$$

- ${\large E_{DFTB}}$ -->  DFTB total energy
- ${\large E_{BS}}$ -->    Band structure energy
- ${\large E_{coul}}$-->   Coulomb interaction
- ${\large E_{rep}}$-->    Repulsive energy 

<hr style="border: 1.5px solid gray;">

## Input and Output Files

**Geometry File (`.gen`)**

<pre style="background-color: #f6f6f6">
5  C    
H  C    
  1 1  0.528800  0.161000  0.93590
  2 2  0.000000  0.000000  0.00000
  3 1  0.205100  0.824000  0.67860
  4 1  0.334500 -0.931400 -0.44960
  5 1 -1.068500 -0.053700  0.19210
</pre>

**Input File (`dftb_in.hsd`)**

<pre style="background-color: #f6f6f6">
Geometry = GenFormat {
   <<< "methane.gen"
}

Hamiltonian = DFTB {
  SCC = Yes
  SlaterKosterFiles {
     H-H = "../../../../H-H.skf"
     C-H = "../../../../C-H.skf"  
     H-C = "../../../../H-C.skf" 
     C-C = "../../../../C-C.skf"
}
MaxAngularMomentum {
     C = "p"  
     H = "s"
  }
} 
</pre>

<hr style="border: 1.5px solid gray;">

**Output Files**

<pre style="background-color: #f6f6f6">
detailed.out # Full calculation log
*.out.gen # Final geometry in GEN
*.out.xyz # Final geometry in XYZ
charges.bin # Mulliken charges
band.out #band/DOS
</pre>

#### Goals of This Hands-on
- Perform 4 types of calculations using 4 different representative systems
- Build and run all simulations from a parent Jupyter notebook
- Visualize and analyze results interactively
#### Tools and dependencies
- DFTB+, ASE, NGLView, NumPy, Matplotlib, OS tools

In [ ]:
import os
from ase import Atoms
from ase.visualize import view
from ase.io import read
import nglview as nv

<hr style="border: 1.5px solid gray;">

## Geometry Optimization

### Triglucose substrate

In [ ]:
structure_1 = read("structures/all4-1_0.gen")
w = nv.show_ase(structure_1)
w.clear_representations()
w.add_ball_and_stick(radius=0.3)
w

### Create Directory for the Calculations

In [ ]:
! pwd

In [ ]:
! mkdir calculations               # create the parent directory for the calculations
! mkdir calculations/GO            # create the GO directory for the geometry optimzation

Navigate to directory for the geometry optimization

In [ ]:
%cd calculations/GO/

In [ ]:
%%writefile dftb_in.hsd

Geometry = GenFormat {
    <<< "../../structures/all4-1_0.gen"
}
Hamiltonian = DFTB {
    SCC = Yes
    ThirdOrderFull = Yes
    HubbardDerivs {
        H = -0.1857
        C = -0.1492
        O = -0.1575
    }
    Mixer = Broyden {
        MixingParameter = 0.1
    }
    MaxSCCIterations = 300
    SlaterKosterFiles = Type2FileNames {
        Prefix = "../../parameters/3ob-3-1/"
        Separator = "-"
        Suffix = ".skf"
    }
MaxAngularMomentum {
    H = s
    C = p
    O = p
}
}
Driver = GeometryOptimization {
  Optimizer = Rational {}
  MovedAtoms = 1:-1               # Move all atoms in the system
  MaxSteps = 200                  # Stop after maximal 100 steps
  OutputPrefix = "geom.out"       # Final geometry in geom.out.{xyz,gen}
  Convergence {GradElem = 0.001}   # Stop if maximal force below 1E-4 H/a0
}

<hr style="border: 1.5px solid gray;">

In [ ]:
# Run calculation
! dftb+ > triglucose.log

check the directory

In [ ]:
! ls

control the log file

In [ ]:
! cat triglucose.log

extract total energies and save into `energies.txt`

In [ ]:
! grep "Total Energy" triglucose.log | awk '{print $3}' > energies.txt

control the `energies.txt` file

In [ ]:
! cat energies.txt

plot the total and relative energy with respect to geometry steps

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Read the log file
with open("triglucose.log", "r") as f:
    lines = f.readlines()

#energy values
energies = pd.read_csv("energies.txt", header=None)
energy_diff = [i - min(energies[0]) for i in energies[0]]


# Plotting
ax1.plot(range(1, len(energies) + 1), energies, marker='*', color='crimson', markersize=10, linestyle='--')
ax1.set_xlabel("Geometry Step")
ax1.set_ylabel("Total Energy (Hartree)", fontsize=14)
ax1.set_title("Total Energy")

ax2.plot(range(1, len(energies) + 1), energy_diff, marker='o', color='goldenrod', markersize=10, linestyle='--')
ax2.set_xlabel("Geometry Step")
ax2.set_ylabel("Relative Energy (Hartree)", fontsize=14)
ax2.set_title("Relative Energy")

_ = fig.suptitle("Energy Convergence During Geometry Optimization")

Now let's look at how the optimized geometry looks

In [ ]:
structure_1_opt = read("geom.out.gen")
w2 = nv.show_ase(structure_1_opt)
w2.clear_representations()
w2.add_ball_and_stick(radius=0.3)
w2

<hr style="border: 1.5px solid gray;">

## HOMO-LUMO GAP

### Triangular Finite $Mo_aS_b$ Nanoparticle

In this section we are going to investigate finite $Mo_{28}S_{72}$ nanoparticle.

In [ ]:
structure_2 = read("../../structures/MoS2_triangle.gen")
w = nv.show_ase(structure_2)
w.clear_representations()
w.add_ball_and_stick(radius=0.5)
w

first lets go back to main directory

In [ ]:
%cd ../..

In [ ]:
! ls

and let's create a directory named `homo_lumo_gap` where we will perform this calculation

In [ ]:
! mkdir calculations/homo_lumo_gap     # create the homo-lumo-gap directory

In [ ]:
%cd calculations/homo_lumo_gap

now we write our input file and save as `dftb_in.hsd`

In [ ]:
%%writefile dftb_in.hsd

Geometry = GenFormat {
    <<< "../../structures/MoS2_triangle.gen"
}

Hamiltonian = DFTB {
    SCC = Yes
    SCCTolerance = 1e-5
    MaxSCCIterations = 1000
    SlaterKosterFiles = Type2FileNames {
        Prefix = "../../parameters/ptbp/"
        Separator = "-"
        Suffix = ".skf"
    }
MaxAngularMomentum {
    Mo = "d"
    S = "p"
}
Charge = 0
Filling = Fermi {
    Temperature [Kelvin] = 300
}
}

Analysis = {
    WriteEigenvectors = Yes
}
Options {
    WriteDetailedXml = Yes
    WriteResultsTag = Yes
}

ParserOptions {
    ParserVersion = 14
}

We run the calculation

In [ ]:
! dftb+ > MoS2_triangle.log

Now we use simple python script to extract HOMO, LUMO and Gap values in eV

In [ ]:
homo = None
lumo = None

with open("band.out") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) != 3:
            continue # skip bad lines
        ev = float(parts[1])
        occ = float(parts[2])
        if occ > 1e-3:
            homo = ev # keep updating until last occupied
        elif lumo is None:
            lumo = ev # first unoccupied eigenvalue

if homo is not None and lumo is not None:
    print(f"HOMO: {homo:.3f} eV")
    print(f"LUMO: {lumo:.3f} eV")
    print(f"Gap : {lumo - homo:.3f} eV")
else:
    print("Could not find HOMO or LUMO.")

<hr style="border: 1.5px solid gray;">

## Density of States and Bandstructure

### Graphene: 4x4x1 Supercell

In this section we are going to investigate density of states and electronic band structure of 4x4 supercell graphene monolayer
<br>
Let's first look how our monolayer looks

In [ ]:
structure_3 = read("../../structures/graphene_441.gen")
w = nv.show_ase(structure_3)
w.clear_representations()
w.add_ball_and_stick(radius=0.35)
w

we go one directory back and create a directory for this section

In [ ]:
%cd ..

In [ ]:
! mkdir dos_bs

In [ ]:
%cd dos_bs/

Firstly we are going to optimize the monolayer, and let's create a directory for that

In [ ]:
!mkdir GO
%cd GO

In [ ]:
%%writefile dftb_in.hsd

Geometry = GenFormat {
    <<< "../../../structures/graphene_441.gen"
}

Hamiltonian = DFTB {
    SCC = Yes
    MaxSCCIterations = 300
    SlaterKosterFiles = Type2FileNames {
        Prefix = "../../../parameters/mio-1-1/"
        Separator = "-"
        Suffix = ".skf"
    }
MaxAngularMomentum {
    C = "p"
}
KPointsAndWeights = SupercellFolding {
  24 0 0
  0 24 0
  0 0 1
  0.0 0.0 0.0
}
Filling = Fermi {
  Temperature [Kelvin] = 100
}
}

Driver = GeometryOptimization {
  Optimizer = Rational {}
  LatticeOpt = Yes
  Isotropic = Yes
  MovedAtoms = 1:-1               # Move all atoms in the system
  MaxSteps = 200                  # Stop after maximal 100 steps
  OutputPrefix = "geom.out"       # Final geometry in geom.out.{xyz,gen}
  Convergence {GradElem = 0.001}   # Stop if maximal force below 1E-4 H/a0
}

In [ ]:
! dftb+ > graphene_opt.log

now since our geometry is optimized we can make DOS calculation. For that purpose, we create a subdirectory called dos

In [ ]:
! mkdir dos
%cd dos

For DOS calculation, we are going to use the optimized geometry. Let's copy it to that folder.

In [ ]:
! cp ../geom.out.gen ./opt.gen

Now we create our input file for the DOS calculation.

In [ ]:
%%writefile dftb_in.hsd

Geometry = GenFormat {
    <<< "opt.gen"
}

Hamiltonian = DFTB {
  Scc = Yes
  SccTolerance = 1e-5
  SlaterKosterFiles = Type2FileNames {
    Prefix = "../../../../parameters/mio-1-1/"
    Separator = "-"
    Suffix = ".skf"
  }
MaxAngularMomentum {
    C = "p"
}
  KPointsAndWeights = SupercellFolding {
    48 0 0
    0 48 0
    0 0  1
    0 0 0
  }
}

Analysis {
  ProjectStates {
    Region {
      Atoms = C
      ShellResolved = Yes
      Label = "dos_C"
    }
  }
}

ParserOptions {
  ParserVersion = 12
}

And we run

In [ ]:
! dftb+ > graphene_dos.log

In order to visualize the density of states and the partial density of states, we should convert the corresponding human readable files (with prefix .out) to XY-format data

In [ ]:
%%bash

dp_dos band.out dos_total.dat
dp_dos -w dos_C.1.out dos_C.1.dat
dp_dos -w dos_C.2.out dos_C.2.dat

now we copy `dos.py` from parent folder to here to use it to plot DOS of graphene.

In [ ]:
! cp ../../../../dos.py .

In [ ]:
from dos import plot_dos

In [ ]:
plot_dos(".", show_total_only=False)

Now we are going to continue for the electronic band structure calculation. First lets create a directory for the band structure calculation.

In [ ]:
! mkdir bs
%cd bs

For the electronic band structure calculation, we need `charges.bin` and optimized geometry file

In [ ]:
! cp ../charges.bin .
! cp ../opt.gen .

In [ ]:
%%writefile dftb_in.hsd

Geometry = GenFormat {
    <<< "opt.gen"
}

Hamiltonian = DFTB {
  Scc = Yes
  ReadInitialCharges = Yes
  MaxSCCIterations = 1
  SccTolerance = 1e-5
  SlaterKosterFiles = Type2FileNames {
    Prefix = "../../../../../parameters/mio-1-1/"
    Separator = "-"
    Suffix = ".skf"
  }
MaxAngularMomentum {
    C = "p"
}
KPointsAndWeights = KLines {
 1    0.0  0.0  0.0                 # Gamma
 15    0.33333333  0.66666666 0.0    # K
 15    0.5  0.0  0.0                 # M
 15    0.0  0.0  0.0                 # Gamma
}
}


ParserOptions {
  ParserVersion = 12
}

Let's run the calculation

In [ ]:
! dftb+ > graphene_bs.log

We again convert `band.out` into `.dat` file in order to post process

In [ ]:
! dp_bands band.out band

And now let's plot the band structure of 4x4 monolayer graphene at path Γ-K-M-Γ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt("band_tot.dat")
k_points = data[:, 0]
bands = data[:, 1:] + 4.67

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for i in range(bands.shape[1]):
    ax.plot(k_points, bands[:, i], color='darkblue', linewidth=2)

ax.axhline(0, color="crimson", linestyle="--", linewidth=2)

for i in [16, 31]:
    plt.plot([i, i], [-3, 3], color='gray', linestyle='--')

ax.set_ylim(-3, 3)
ax.set_xlim(1, 46)

_ = ax.set_xticks([1, 16, 31, 46])
_ = ax.set_xticklabels(['Γ', 'K', 'M', 'Γ'], fontsize=13)

_ = ax.set_ylabel('Energy (eV)', fontsize=16)